In [ ]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer


# Define IAM role
role = get_execution_role()
prefix = <prefix>
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

In [ ]:
bucket = <backetname>

In [ ]:
df_train = pd.read_csv('./sage_maker_train.csv')

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train = df_train.drop(['Name','Cabin'],axis=1)

In [ ]:
df_tain = df_train.drop('Ticket',axis=1)

In [ ]:
df_train = df_tain.drop('Fare',axis=1)

In [ ]:
df_train.to_csv('train.csv', index=False, header=False)

In [ ]:
sess = sagemaker.Session()
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

In [ ]:

df_train["Sex"] = df_train["Sex"].map({"male":0,"female":1})

In [ ]:
df_train = pd.get_dummies(df_train,columns=["Embarked"])

In [ ]:
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [ ]:
xgb.fit({'train': s3_input_train})

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')